# Анализ разных метрик тематической близости документов

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
from scipy.linalg import norm
from scipy.stats import entropy
from pymongo import MongoClient
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

In [3]:
client = MongoClient()
collection = client["datasets"]["postnauka"]

In [4]:
def t(doc_id):
    return collection.find_one("pn_%d" % doc_id)["title"]

In [5]:
artm_model = pickle.load(open("../server/hartm.mdl", "rb"))

In [6]:
artm_model["theta"].shape

(110, 3446)

In [7]:
theta_lvl0 = artm_model["theta"][:10].T

In [8]:
theta_lvl1 = artm_model["theta"][10:-70].T

In [9]:
theta_lvl2 = artm_model["theta"][-70:].T

## Близость тематических профилей (косинусная мера)

In [10]:
doc_id = 3123

In [11]:
# Первый уровень

In [12]:
theta_lvl = theta_lvl0.sort_index()
print("> %s (doc_id=%d)\n" % (t(doc_id), doc_id))
sim_matrix = cosine_similarity([theta_lvl.loc[doc_id]], theta_lvl)
print("Top-5 similar (p(t|d) cosine similarity) documents:")
for rid, (prob, sim_doc_id) in enumerate(zip(np.sort(sim_matrix)[0, -11:-1][::-1], np.argsort(sim_matrix)[0, -11:-1][::-1])):
    print("%d. %s (doc_id=%d, p=%.4f)" % (rid + 1, t(sim_doc_id + 1), sim_doc_id + 1, prob))

> Математические методы прогнозирования объемов продаж (doc_id=3123)

Top-5 similar (p(t|d) cosine similarity) documents:
1. Главы | Закономерности простых чисел. Гипотеза Римана (doc_id=2109, p=0.9985)
2. По шагам | Теория принятия решений (doc_id=3423, p=0.9976)
3. 5 книг о поведенческой экономике (doc_id=3344, p=0.9965)
4. Психология создания трудностей и проблем (doc_id=2988, p=0.9949)
5. Что такое «робот»? (doc_id=2296, p=0.9943)
6. Курс «Теория принятия решений: математические модели выбора» (doc_id=3181, p=0.9939)
7. Задачи и проблемы в мышлении (doc_id=1665, p=0.9936)
8. Эмоциональные вычисления (doc_id=2295, p=0.9933)
9. Марвин Мински и эмоциональные машины (doc_id=3069, p=0.9928)
10. Дэвид Вернон: «То, что мы называем искусственным интеллектом, им не является» (doc_id=2256, p=0.9927)


## Близость тематических профилей (KL-дивергенция)

In [13]:
doc_id = 3123

In [14]:
def sym_kl(p, q):
    return entropy(p, q) + entropy(q, p)

In [15]:
# Первый уровень

In [16]:
%%time

theta_lvl = theta_lvl0.sort_index()
print("> %s (doc_id=%d)\n" % (t(doc_id), doc_id))
sim_matrix = pairwise_distances([theta_lvl.loc[doc_id]], theta_lvl, sym_kl, n_jobs=-1)
print("Top-10 similar (p(t|d) symmetric KL-divergence) documents:")
for rid, (prob, sim_doc_id) in enumerate(zip(np.sort(sim_matrix)[0, 1:11], np.argsort(sim_matrix)[0, 1:11])):
    print("%d. %s (doc_id=%d, p=%.4f)" % (rid + 1, t(sim_doc_id + 1), sim_doc_id + 1, prob))

> Математические методы прогнозирования объемов продаж (doc_id=3123)

Top-10 similar (p(t|d) symmetric KL-divergence) documents:
1. Главы | Закономерности простых чисел. Гипотеза Римана (doc_id=2109, p=0.0967)
2. Главы | Методы измерения данных (doc_id=1051, p=0.1277)
3. Марвин Мински и эмоциональные машины (doc_id=3069, p=0.1452)
4. Построение сложных вероятностных моделей (doc_id=2892, p=0.1529)
5. Эмоциональные вычисления (doc_id=2295, p=0.1542)
6. Психология создания трудностей и проблем (doc_id=2988, p=0.1583)
7. FAQ: Компьютерные доказательства (doc_id=1121, p=0.1693)
8. 5 книг о поведенческой экономике (doc_id=3344, p=0.2286)
9. По шагам | Теория принятия решений (doc_id=3423, p=0.2335)
10. Исследования мышления в когнитивной психологии (doc_id=2947, p=0.2378)
CPU times: user 64 ms, sys: 28 ms, total: 92 ms
Wall time: 290 ms


## Близость тематических профилей (расстояние Хеллингера)

In [17]:
doc_id = 3123

In [18]:
def hellinger_dist(p, q):
    return norm(np.sqrt(p) - np.sqrt(q))

In [19]:
# Первый уровень

In [20]:
theta_lvl = theta_lvl0.sort_index()
print("> %s (doc_id=%d)\n" % (t(doc_id), doc_id))
sim_matrix = pairwise_distances([theta_lvl.loc[doc_id]], theta_lvl, hellinger_dist)
print("Top-10 similar (p(t|d) hellinger distance) documents:")
for rid, (prob, sim_doc_id) in enumerate(zip(np.sort(sim_matrix)[0, 1:11], np.argsort(sim_matrix)[0, 1:11])):
    print("%d. %s (doc_id=%d, p=%.4f)" % (rid + 1, t(sim_doc_id + 1), sim_doc_id + 1, prob))

> Математические методы прогнозирования объемов продаж (doc_id=3123)

Top-10 similar (p(t|d) hellinger distance) documents:
1. Главы | Закономерности простых чисел. Гипотеза Римана (doc_id=2109, p=0.1408)
2. Главы | Методы измерения данных (doc_id=1051, p=0.1643)
3. Психология создания трудностей и проблем (doc_id=2988, p=0.1798)
4. Построение сложных вероятностных моделей (doc_id=2892, p=0.1827)
5. Марвин Мински и эмоциональные машины (doc_id=3069, p=0.1861)
6. Эмоциональные вычисления (doc_id=2295, p=0.1918)
7. FAQ: Компьютерные доказательства (doc_id=1121, p=0.1927)
8. По шагам | Теория принятия решений (doc_id=3423, p=0.2098)
9. 5 книг о поведенческой экономике (doc_id=3344, p=0.2142)
10. «Разработка операционной системы рыночного уровня должна укладываться в 1 млрд долларов» (doc_id=1734, p=0.2190)


## Анализ тематических профилей

In [21]:
theta_lvl = theta_lvl1.sort_index()

In [22]:
theta_lvl.loc[3123]

level1_topic_0     0.006610
level1_topic_1     0.000028
level1_topic_2     0.000386
level1_topic_3     0.000023
level1_topic_4     0.009620
level1_topic_5     0.001744
level1_topic_6     0.026533
level1_topic_7     0.000150
level1_topic_8     0.048912
level1_topic_9     0.000553
level1_topic_10    0.000025
level1_topic_11    0.000033
level1_topic_12    0.001444
level1_topic_13    0.000027
level1_topic_14    0.000031
level1_topic_15    0.033075
level1_topic_16    0.280329
level1_topic_17    0.051039
level1_topic_18    0.302440
level1_topic_19    0.153228
level1_topic_20    0.000022
level1_topic_21    0.000023
level1_topic_22    0.027129
level1_topic_23    0.027075
level1_topic_24    0.006719
level1_topic_25    0.000036
level1_topic_26    0.009875
level1_topic_27    0.000219
level1_topic_28    0.004038
level1_topic_29    0.008635
Name: 3123, dtype: float32

In [23]:
theta_lvl.loc[2257]

level1_topic_0     0.009424
level1_topic_1     0.000032
level1_topic_2     0.001288
level1_topic_3     0.000033
level1_topic_4     0.017803
level1_topic_5     0.002538
level1_topic_6     0.008913
level1_topic_7     0.000083
level1_topic_8     0.053283
level1_topic_9     0.001030
level1_topic_10    0.003718
level1_topic_11    0.000185
level1_topic_12    0.000032
level1_topic_13    0.000021
level1_topic_14    0.001256
level1_topic_15    0.027246
level1_topic_16    0.258919
level1_topic_17    0.028847
level1_topic_18    0.353488
level1_topic_19    0.153007
level1_topic_20    0.000026
level1_topic_21    0.000025
level1_topic_22    0.005661
level1_topic_23    0.032550
level1_topic_24    0.013211
level1_topic_25    0.000029
level1_topic_26    0.012926
level1_topic_27    0.002171
level1_topic_28    0.006999
level1_topic_29    0.005257
Name: 2257, dtype: float32

In [24]:
theta_lvl.loc[1734]

level1_topic_0     0.010725
level1_topic_1     0.000212
level1_topic_2     0.000017
level1_topic_3     0.000021
level1_topic_4     0.019576
level1_topic_5     0.007783
level1_topic_6     0.001676
level1_topic_7     0.002375
level1_topic_8     0.013219
level1_topic_9     0.000790
level1_topic_10    0.000021
level1_topic_11    0.000479
level1_topic_12    0.001415
level1_topic_13    0.000022
level1_topic_14    0.000086
level1_topic_15    0.124689
level1_topic_16    0.253682
level1_topic_17    0.022450
level1_topic_18    0.295885
level1_topic_19    0.126896
level1_topic_20    0.000022
level1_topic_21    0.000712
level1_topic_22    0.000017
level1_topic_23    0.039969
level1_topic_24    0.023700
level1_topic_25    0.001273
level1_topic_26    0.008254
level1_topic_27    0.005643
level1_topic_28    0.017148
level1_topic_29    0.021242
Name: 1734, dtype: float32

---